Credits :
- [[PART 2] - RAPIDS TfidfVectorizer - [CV 0.700]](https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700)
- [🛒 Shopee: EDA 📊 preprocessing ⚙️ pl](https://www.kaggle.com/prokaggler/shopee-eda-preprocessing-pytorch-bert/comments#-Data-Preprocessing-)
- [EDA And Text Embedding](https://www.kaggle.com/marwanelghitany/eda-and-text-embedding/comments)

In [ ]:
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora import Dictionary
from gensim.models import Word2Vec, WordEmbeddingSimilarityIndex
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
test  = pd.read_csv('../input/shopee-product-matching/test.csv')

len_test = len(test)
len_pred = len_test if len_test > 3 else len(train)
COMPUTE_CV = False if len_test > 3 else True

### Preprocess

In [ ]:
%%time
def tokenize(text):
    return text.split(' ')

def preprocess_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence = sentence.split()
    sentence = [word for word in sentence if word not in stopwords.words('english')]
    wl = WordNetLemmatizer()
    sentence = [wl.lemmatize(word) for word in sentence]
    sentence = ' '.join(sentence)
    return sentence

train['title_clean'] = train['title'].progress_apply(preprocess_text)
test['title_clean']  = test['title'].progress_apply(preprocess_text)

df = pd.concat([test, train]).reset_index(drop=True) if len_test > 3 else train

### Word2Vec

In [ ]:
%%time
# bag of words
titles_tokens = [title.split() for title in df['title_clean']]

# word dictionary
dictionary = Dictionary(titles_tokens)
# dictionary.filter_extremes(no_below=2, no_above=50)

# Bag of words
bow_corpus = [dictionary.doc2bow(token) for token in titles_tokens]

In [ ]:
%%time
model = Word2Vec(titles_tokens, size=50, window=5, min_count=1, workers=4)

In [ ]:
%%time
# word similarity index
termsim_index = WordEmbeddingSimilarityIndex(model.wv)

# similarity matrix
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)

# soft cosine similarity
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=50)

In [ ]:
def getFilteredQuery(query, dict):
    query = query.split()
    return([wrd for wrd in query if dict.doc2idx([wrd])[0] >= 0])

def getIndex(sim, boundry, threshold):
    inds = [s[0] for s in sim if s[1] > threshold]
    if boundry > 3:
        inds = [i for i in inds if i < boundry]
    return inds

def getSimilarDoc(sentences, dict, docsim_index, boundry, threshold = 0.7):
    queries = [dict.doc2bow(getFilteredQuery(sent, dict)) for sent in sentences]
    sims = docsim_index[queries]
    return [getIndex(sim, boundry, threshold) for sim in sims]

In [ ]:
preds = []
CHUNK = 1024 * 4

print('Finding similar titles...')
CTS = len_pred // CHUNK
if len_pred % CHUNK != 0: CTS += 1
for j in range(CTS):
    
    a, b = j * CHUNK, min((j+1) * CHUNK, len_pred)
    print('chunk',a,'to',b)
    
    # get similar sentences
    index = getSimilarDoc(df['title_clean'][a:b], dictionary, docsim_index, len_test, 0.98)
    
    for k in range(b-a):
        o = df.iloc[index[k]].posting_id.values
        preds.append(o)

In [ ]:
if len(df) > len_pred:
    df = df[:len_pred]
df['preds'] = preds
df.head()

### Submission

In [ ]:
def combine_for_sub(row):
    x = np.concatenate([row.preds])
    return ' '.join(np.unique(x))

df['matches'] = df.apply(combine_for_sub,axis=1)

In [ ]:
df[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()